In [1]:
import pandas as pd
import altair as alt

from datetime import timedelta
import pytz

import sys
sys.path.append('../')
from mod import load_data, alt_theme

In [2]:
comments, issues, prs = load_data('../../data/')
bot_names = pd.read_csv('../bot_names.csv')
comments = comments.query('author not in @bot_names')
issues = issues.query('author not in @bot_names')
prs = prs.query('author not in @bot_names')

In [3]:
renderer = "jupyterlab"

In [4]:
# Parameters
renderer = "kaggle"


In [5]:
alt.renderers.enable(renderer);
alt.themes.register('my_theme', alt_theme)
alt.themes.enable("my_theme")

ThemeRegistry.enable('my_theme')

In [6]:
use_data = prs  # comments
seen_times = use_data.groupby(['org', 'author']).agg({'updatedAt': ['min', 'max']})['updatedAt']

In [7]:
seen_times = seen_times.rename(columns={'min': 'first_seen', 'max': 'last_seen'})
for col in seen_times.columns:
    seen_times[col] = pd.to_datetime(seen_times[col], utc=True)

In [8]:
# "recent" defined as N days in the past
recently = pd.datetime.utcnow() - timedelta(days=30)
recently = pytz.utc.localize(recently)

charts = []
for org, idata in seen_times.groupby('org'):
    idata = idata.loc[org]

    idata['seen_recently'] = idata.eval('last_seen > @recently')


    first_seen = idata.resample('M', on='first_seen').agg({'first_seen': 'count', 'seen_recently': 'sum'})
    first_seen.index.name = 'date'

    first_seen = pd.melt(first_seen.reset_index(), id_vars='date', var_name='kind', value_name='count')

    first_seen['date_name'] = first_seen['date'].map(lambda a: "{:%Y-%m}".format(a))

    bar = alt.Chart(first_seen, title=org).mark_bar().encode(
        x='count:Q',
        y='kind',
        color='kind',
        row='date_name:O'
    )
    charts.append(bar)
    
alt.hconcat(*charts)

/home/choldgraf/anaconda/envs/dev/lib/python3.6/site-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


HConcatChart({
  hconcat: [Chart({
    data:                         date           kind  count date_name
    0  2017-10-31 00:00:00+00:00     first_seen      3   2017-10
    1  2017-11-30 00:00:00+00:00     first_seen      2   2017-11
    2  2017-12-31 00:00:00+00:00     first_seen      0   2017-12
    3  2018-01-31 00:00:00+00:00     first_seen      0   2018-01
    4  2018-02-28 00:00:00+00:00     first_seen      1   2018-02
    5  2018-03-31 00:00:00+00:00     first_seen      0   2018-03
    6  2018-04-30 00:00:00+00:00     first_seen      0   2018-04
    7  2018-05-31 00:00:00+00:00     first_seen      0   2018-05
    8  2018-06-30 00:00:00+00:00     first_seen      0   2018-06
    9  2018-07-31 00:00:00+00:00     first_seen      1   2018-07
    10 2018-08-31 00:00:00+00:00     first_seen      2   2018-08
    11 2018-09-30 00:00:00+00:00     first_seen      2   2018-09
    12 2018-10-31 00:00:00+00:00     first_seen      4   2018-10
    13 2018-11-30 00:00:00+00:00     first_seen      2   2018-11
    14 2018-12-31 00:00:00+00:00     first_seen      1   2018-12
    15 2019-01-31 00:00:00+00:00     first_seen      0   2019-01
    16 2019-02-28 00:00:00+00:00     first_seen      2   2019-02
    17 2019-03-31 00:00:00+00:00     first_seen      3   2019-03
    18 2019-04-30 00:00:00+00:00     first_seen      0   2019-04
    19 2019-05-31 00:00:00+00:00     first_seen      4   2019-05
    20 2017-10-31 00:00:00+00:00  seen_recently  False   2017-10
    21 2017-11-30 00:00:00+00:00  seen_recently  False   2017-11
    22 2017-12-31 00:00:00+00:00  seen_recently  False   2017-12
    23 2018-01-31 00:00:00+00:00  seen_recently  False   2018-01
    24 2018-02-28 00:00:00+00:00  seen_recently  False   2018-02
    25 2018-03-31 00:00:00+00:00  seen_recently  False   2018-03
    26 2018-04-30 00:00:00+00:00  seen_recently  False   2018-04
    27 2018-05-31 00:00:00+00:00  seen_recently  False   2018-05
    28 2018-06-30 00:00:00+00:00  seen_recently  False   2018-06
    29 2018-07-31 00:00:00+00:00  seen_recently  False   2018-07
    30 2018-08-31 00:00:00+00:00  seen_recently  False   2018-08
    31 2018-09-30 00:00:00+00:00  seen_recently   True   2018-09
    32 2018-10-31 00:00:00+00:00  seen_recently  False   2018-10
    33 2018-11-30 00:00:00+00:00  seen_recently  False   2018-11
    34 2018-12-31 00:00:00+00:00  seen_recently  False   2018-12
    35 2019-01-31 00:00:00+00:00  seen_recently  False   2019-01
    36 2019-02-28 00:00:00+00:00  seen_recently  False   2019-02
    37 2019-03-31 00:00:00+00:00  seen_recently  False   2019-03
    38 2019-04-30 00:00:00+00:00  seen_recently  False   2019-04
    39 2019-05-31 00:00:00+00:00  seen_recently  False   2019-05,
    encoding: FacetedEncoding({
      color: Color({
        field: 'kind',
        type: 'nominal'
      }),
      row: Row({
        field: 'date_name',
        type: 'ordinal'
      }),
      x: X({
        field: 'count',
        type: 'quantitative'
      }),
      y: Y({
        field: 'kind',
        type: 'nominal'
      })
    }),
    mark: 'bar',
    title: 'binder-examples'
  }), Chart({
    data:                          date           kind  count date_name
    0   2015-01-31 00:00:00+00:00     first_seen   27.0   2015-01
    1   2015-02-28 00:00:00+00:00     first_seen   11.0   2015-02
    2   2015-03-31 00:00:00+00:00     first_seen   30.0   2015-03
    3   2015-04-30 00:00:00+00:00     first_seen    6.0   2015-04
    4   2015-05-31 00:00:00+00:00     first_seen    4.0   2015-05
    5   2015-06-30 00:00:00+00:00     first_seen    6.0   2015-06
    6   2015-07-31 00:00:00+00:00     first_seen    9.0   2015-07
    7   2015-08-31 00:00:00+00:00     first_seen   25.0   2015-08
    8   2015-09-30 00:00:00+00:00     first_seen   11.0   2015-09
    9   2015-10-31 00:00:00+00:00     first_seen   15.0   2015-10
    10  2015-11-30 00:00:00+00:00     first_seen   10.0   2015-11
    11  2015-12-31 00:00:00+00:00     first_seen    7.0   2015-12
    12  2016-01-31 00:00:00

In [9]:
%%html
<script src="https://cdn.rawgit.com/parente/4c3e6936d0d7a46fd071/raw/65b816fb9bdd3c28b4ddf3af602bfd6015486383/code_toggle.js"></script>